# Overview
This script aims to pridict list of mitre attack technique given a techinque to detect. This will help to give the use a means to know which techniques run following a script that was choses. Eg. User picks T1027 and runs the sript. if the technique is detected in the log, then he is recommended with a list of other techniques to run a search on. 

The algoritm follows a statistical approach where it finds the most number of times the technique were used togather. The dataset used for this statistical analysis is done by using APT actors from MITRE Attack repository. Once we get the common pairs, we form a weighted graph and then using nearest neighbour finder to find the list of techniques that is related to each other.

In [1]:
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from pprint import pprint
from collections import Counter
from pyattck import Attck
import networkx as nx

In [2]:
# Init pyatt lib
attck = Attck()

co-occurrance matrix - https://en.wikipedia.org/wiki/Co-occurrence_matrix#:~:text=A%20co%2Doccurrence%20matrix%20or,colors)%20at%20a%20given%20offset.

In [3]:
# Gather and store edge pairs using co-occurrance matrix
sourcelist = []
targetlist = []
target_tacticlist = []

for actor in attck.enterprise.actors:
    edge_i = [x.id for x in actor.techniques]
    edge_j = [x.id for x in actor.techniques]
    
    for technique_i in edge_i:
        for technique_j in edge_j:
            if technique_j != technique_i:
                sourcelist.append(technique_i)
                targetlist.append(technique_j)

In [4]:
# User counter to count number of time edges appear to create weights
counter = Counter(tuple(sorted(tup)) for tup in zip(sourcelist,targetlist))
counter

Counter({('T1027', 'T1566.001'): 76,
         ('T1053.005', 'T1566.001'): 50,
         ('T1204.002', 'T1566.001'): 110,
         ('T1566.001', 'T1571'): 14,
         ('T1036.004', 'T1566.001'): 18,
         ('T1059.005', 'T1566.001'): 60,
         ('T1105', 'T1566.001'): 76,
         ('T1027', 'T1053.005'): 56,
         ('T1027', 'T1204.002'): 78,
         ('T1027', 'T1571'): 18,
         ('T1027', 'T1036.004'): 22,
         ('T1027', 'T1059.005'): 54,
         ('T1027', 'T1105'): 86,
         ('T1053.005', 'T1204.002'): 48,
         ('T1053.005', 'T1571'): 12,
         ('T1036.004', 'T1053.005'): 18,
         ('T1053.005', 'T1059.005'): 36,
         ('T1053.005', 'T1105'): 54,
         ('T1204.002', 'T1571'): 14,
         ('T1036.004', 'T1204.002'): 20,
         ('T1059.005', 'T1204.002'): 60,
         ('T1105', 'T1204.002'): 76,
         ('T1036.004', 'T1571'): 8,
         ('T1059.005', 'T1571'): 16,
         ('T1105', 'T1571'): 18,
         ('T1036.004', 'T1059.005'): 12,
         (

In [55]:
# Store each value in array list
sourcelist = []
targetlist = []
weightlist = []
for key, value in counter.items():
    sourcelist.append(key[0])
    targetlist.append(key[1])
    weightlist.append(value)
    
# check if the len are same value
print(len(sourcelist))
print(len(targetlist))
print(len(weightlist))
# standardising weights, high value = less likely pairs
weightlist = [1-i/max(weightlist) for i in weightlist]

# TODO: using total techniques

19201
19201
19201


In [56]:
# Store the nodes and weights in dataframe
pd.set_option('display.max_rows', 20)
edges = pd.DataFrame(
    {
        "source": sourcelist,#sourceId,
        "target": targetlist, #targetlist
        "weight": weightlist
    }
)
edges

,source,target,weight
0,T1027,T1566.001,0.309091
1,T1053.005,T1566.001,0.545455
2,T1204.002,T1566.001,0.000000
3,T1566.001,T1571,0.872727
4,T1036.004,T1566.001,0.836364
...,...,...,...
19196,T1003.003,T1568.001,0.981818
19197,T1106,T1583.001,0.981818
19198,T1106,T1568.001,0.981818
19199,T1083,T1568.001,0.981818


In [57]:
# Create a graph using the dataframe
G = nx.from_pandas_edgelist(edges, edge_attr=True, create_using=nx.DiGraph())

In [33]:
tacticcolumns = {}
for tactic in attck.enterprise.tactics:
    templist = []
    for teachniques in tactic.techniques:
        templist.append(teachniques.id)
    tacticcolumns[tactic.id] = sorted(templist)

import collections
tacticcolumns = collections.OrderedDict(sorted(tacticcolumns.items()))
pd.set_option('display.max_rows', None)
df = pd.DataFrame.from_dict(tacticcolumns, orient='index')
df = df.fillna("-")
df = df.transpose()
df

,TA0001,TA0002,TA0003,TA0004,TA0005,TA0006,TA0007,TA0008,TA0009,TA0010,TA0011,TA0040,TA0042,TA0043
0,T1078,T1047,T1034,T1034,T1006,T1003,T1007,T1021,T1005,T1011,T1001,T1485,T1583,T1589
1,T1078.001,T1053,T1037,T1037,T1014,T1003.001,T1010,T1021.001,T1025,T1011.001,T1001.001,T1486,T1583.001,T1589.001
2,T1078.002,T1053.001,T1037.001,T1037.001,T1027,T1003.002,T1012,T1021.002,T1039,T1020,T1001.002,T1489,T1583.002,T1589.002
3,T1078.003,T1053.002,T1037.002,T1037.002,T1027.001,T1003.003,T1016,T1021.003,T1056,T1020.001,T1001.003,T1490,T1583.003,T1589.003
4,T1078.004,T1053.003,T1037.003,T1037.003,T1027.002,T1003.004,T1016.001,T1021.004,T1056.001,T1029,T1008,T1491,T1583.004,T1590
5,T1091,T1053.004,T1037.004,T1037.004,T1027.003,T1003.005,T1018,T1021.005,T1056.002,T1030,T1026,T1491.001,T1583.005,T1590.001
6,T1133,T1053.005,T1037.005,T1037.005,T1027.004,T1003.006,T1033,T1021.006,T1056.003,T1041,T1043,T1491.002,T1583.006,T1590.002
7,T1189,T1053.006,T1053,T1053,T1027.005,T1003.007,T1040,T1051,T1056.004,T1048,T1071,T1495,T1584,T1590.003
8,T1190,T1053.007,T1053.001,T1053.001,T1036,T1003.008,T1046,T1072,T1074,T1048.001,T1071.001,T1496,T1584.001,T1590.004
9,T1195,T1059,T1053.002,T1053.002,T1036.001,T1040,T1049,T1080,T1074.001,T1048.002,T1071.002,T1498,T1584.002,T1590.005


# Shortest path from inversed weighted graph

In [34]:
# get all shortest path from a node to unknown target
startnode = 'T1001'
path = nx.algorithms.shortest_path(G, startnode)
path

{'T1189': ['T1189'],
 'T1016': ['T1189', 'T1016'],
 'T1204.002': ['T1189', 'T1204.002'],
 'T1071.001': ['T1189', 'T1071.001'],
 'T1043': ['T1189', 'T1043'],
 'T1132.001': ['T1189', 'T1132.001'],
 'T1218.010': ['T1189', 'T1218.010'],
 'T1059': ['T1189', 'T1059'],
 'T1082': ['T1189', 'T1082'],
 'T1218.011': ['T1189', 'T1218.011'],
 'T1547.001': ['T1189', 'T1547.001'],
 'T1033': ['T1189', 'T1033'],
 'T1574.002': ['T1189', 'T1574.002'],
 'T1112': ['T1189', 'T1112'],
 'T1059.001': ['T1189', 'T1059.001'],
 'T1566.001': ['T1189', 'T1566.001'],
 'T1543.003': ['T1189', 'T1543.003'],
 'T1140': ['T1189', 'T1140'],
 'T1027': ['T1189', 'T1027'],
 'T1564.003': ['T1189', 'T1564.003'],
 'T1072': ['T1189', 'T1072'],
 'T1505.003': ['T1189', 'T1505.003'],
 'T1068': ['T1189', 'T1068'],
 'T1550.002': ['T1189', 'T1550.002'],
 'T1564.004': ['T1189', 'T1564.004'],
 'T1018': ['T1189', 'T1018'],
 'T1070.006': ['T1189', 'T1070.006'],
 'T1070.001': ['T1189', 'T1070.001'],
 'T1027.001': ['T1189', 'T1027.001'],
 'T

In [58]:
import itertools

startnode = 'T1003'
[list(cc) for cc in nx.strongly_connected_components(G)]

[['T1608.004'],
 ['T1608.001'],
 ['T1608.005'],
 ['T1598.003'],
 ['T1592.004'],
 ['T1590.005'],
 ['T1590'],
 ['T1598'],
 ['T1595.002'],
 ['T1594'],
 ['T1593'],
 ['T1592.002'],
 ['T1591.002'],
 ['T1590.001'],
 ['T1589.003'],
 ['T1589.002'],
 ['T1589'],
 ['T1606.002'],
 ['T1606.001'],
 ['T1587.003'],
 ['T1588.004'],
 ['T1588.006'],
 ['T1588.002'],
 ['T1588.001'],
 ['T1587.001'],
 ['T1589.001'],
 ['T1587'],
 ['T1586.002'],
 ['T1585.002'],
 ['T1585.001'],
 ['T1584.001'],
 ['T1585'],
 ['T1584.006'],
 ['T1584.004'],
 ['T1584.003'],
 ['T1583.006'],
 ['T1583.004'],
 ['T1583.001'],
 ['T1574.006'],
 ['T1587.002'],
 ['T1598.002'],
 ['T1608.002'],
 ['T1574.002'],
 ['T1573.001'],
 ['T1583.003'],
 ['T1571'],
 ['T1567'],
 ['T1568'],
 ['T1573.002'],
 ['T1572'],
 ['T1588.003'],
 ['T1570'],
 ['T1574.001'],
 ['T1574.012'],
 ['T1569.002'],
 ['T1567.002'],
 ['T1566.003'],
 ['T1568.001'],
 ['T1566.002'],
 ['T1568.003'],
 ['T1568.002'],
 ['T1573'],
 ['T1566.001'],
 ['T1564.004'],
 ['T1564.003'],
 ['T1564.001

In [10]:
print("total combinations: ",len(path)-1)
print("smallest paths: ", min([len(x) for x in list(path.values())[1:]]))
print("Largest no of path found: ", max([len(x) for x in path.values()]))
print("Largest path samples: ", list(path.values())[-2:])

total combinations:  338
smallest paths:  2
Largest no of path found:  4
Largest path samples:  [['T1189', 'T1120', 'T1564.005'], ['T1189', 'T1016', 'T1090.001', 'T1556.002']]


In [28]:
selectedpath = 'T1556.002'
def create_colors(x):
    #copy df to new - original data are not changed
    df1 = x.copy()
    #select all values to default value - no color
    df1.loc[:,:] = 'background-color: '
    #overwrite values with green and red color
    for i in path[selectedpath]:
        valuehighlight = i
        getcell = [(df[col][df[col].eq(valuehighlight)].index[i], col) for col in df.columns for i in range(len(df[col][df[col].eq(valuehighlight)].index))]
        getcell = getcell[0]
        df1.loc[getcell[0], getcell[1]] = 'background-color: green'
    
    return df1      

df.style.apply(create_colors, axis=None)

,TA0001,TA0002,TA0003,TA0004,TA0005,TA0006,TA0007,TA0008,TA0009,TA0010,TA0011,TA0040,TA0042,TA0043
0,T1078,T1047,T1034,T1034,T1006,T1003,T1007,T1021,T1005,T1011,T1001,T1485,T1583,T1589
1,T1078.001,T1053,T1037,T1037,T1014,T1003.001,T1010,T1021.001,T1025,T1011.001,T1001.001,T1486,T1583.001,T1589.001
2,T1078.002,T1053.001,T1037.001,T1037.001,T1027,T1003.002,T1012,T1021.002,T1039,T1020,T1001.002,T1489,T1583.002,T1589.002
3,T1078.003,T1053.002,T1037.002,T1037.002,T1027.001,T1003.003,T1016,T1021.003,T1056,T1020.001,T1001.003,T1490,T1583.003,T1589.003
4,T1078.004,T1053.003,T1037.003,T1037.003,T1027.002,T1003.004,T1016.001,T1021.004,T1056.001,T1029,T1008,T1491,T1583.004,T1590
5,T1091,T1053.004,T1037.004,T1037.004,T1027.003,T1003.005,T1018,T1021.005,T1056.002,T1030,T1026,T1491.001,T1583.005,T1590.001
6,T1133,T1053.005,T1037.005,T1037.005,T1027.004,T1003.006,T1033,T1021.006,T1056.003,T1041,T1043,T1491.002,T1583.006,T1590.002
7,T1189,T1053.006,T1053,T1053,T1027.005,T1003.007,T1040,T1051,T1056.004,T1048,T1071,T1495,T1584,T1590.003
8,T1190,T1053.007,T1053.001,T1053.001,T1036,T1003.008,T1046,T1072,T1074,T1048.001,T1071.001,T1496,T1584.001,T1590.004
9,T1195,T1059,T1053.002,T1053.002,T1036.001,T1040,T1049,T1080,T1074.001,T1048.002,T1071.002,T1498,T1584.002,T1590.005


In [27]:
found = ['t1075', 't1550.002', 't1078', 't1190', 't1133', 't1099', 't1070.006', 't1200', 't1136', 't1136.001', 't1003.002', 't1006', 't1084', 't1546.003']

found = [x.replace('t','T') for x in found]
found

['T1075',
 'T1550.002',
 'T1078',
 'T1190',
 'T1133',
 'T1099',
 'T1070.006',
 'T1200',
 'T1136',
 'T1136.001',
 'T1003.002',
 'T1006',
 'T1084',
 'T1546.003']